In [7]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets, models
import torch.nn as nn
import numpy as np
import random
import json
from sklearn.metrics import precision_score, recall_score, f1_score

# Set seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Preprocessing
image_size = 224
batch_size = 32  # Increased batch size for faster training

data_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),  # reduce rotation angle
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),  # reduce heavy jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load Dataset
data_path = 'C:/Users/joeli/OneDrive/Documents/PYTHON/GuviProject5(DL)/Faulty_solar_panel'
full_dataset = datasets.ImageFolder(root=data_path, transform=data_transform)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Save class names
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

# Split Dataset
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Model Setup
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Early Stopping settings
patience = 5
best_accuracy = 0.0
counter = 0

# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Evaluation
    model.eval()
    correct = total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Compute metrics
    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro') * 100
    recall = recall_score(all_labels, all_preds, average='macro') * 100
    f1 = f1_score(all_labels, all_preds, average='macro') * 100

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}")
    print(f"Accuracy: {accuracy:.2f}% | Precision: {precision:.2f}% | Recall: {recall:.2f}% | F1-Score: {f1:.2f}%")

    # Save best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        counter = 0
        torch.save(model.state_dict(), 'best_resnet18_model.pth')
    else:
        counter += 1

    # Early Stopping
    if counter >= patience:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

print(f"Training finished. Best Accuracy: {best_accuracy:.2f}%")


Using device: cpu
Classes: ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']
Epoch [1/20], Loss: 30.0742
Accuracy: 35.59% | Precision: 50.79% | Recall: 34.15% | F1-Score: 29.48%
Epoch [2/20], Loss: 23.9097
Accuracy: 46.33% | Precision: 56.56% | Recall: 41.55% | F1-Score: 41.12%
Epoch [3/20], Loss: 16.8483
Accuracy: 64.97% | Precision: 68.53% | Recall: 64.65% | F1-Score: 65.10%
Epoch [4/20], Loss: 12.3649
Accuracy: 68.36% | Precision: 72.08% | Recall: 69.09% | F1-Score: 68.45%
Epoch [5/20], Loss: 13.5645
Accuracy: 74.01% | Precision: 80.12% | Recall: 71.23% | F1-Score: 73.71%
Epoch [6/20], Loss: 11.3380
Accuracy: 79.10% | Precision: 79.61% | Recall: 77.55% | F1-Score: 78.35%
Epoch [7/20], Loss: 9.0492
Accuracy: 72.32% | Precision: 71.62% | Recall: 69.22% | F1-Score: 67.17%
Epoch [8/20], Loss: 8.1529
Accuracy: 69.49% | Precision: 81.35% | Recall: 64.11% | F1-Score: 67.45%
Epoch [9/20], Loss: 10.0377
Accuracy: 67.80% | Precision: 70.07% | Recall: 66.0